In [1]:
%%bash
echo "TODAY'S DATE:"
date
echo "------------"
echo ""
#Display operating system info
lsb_release -a
echo ""
echo "------------"
echo "HOSTNAME: "; hostname 
echo ""
echo "------------"
echo "Computer Specs:"
echo ""
lscpu
echo ""
echo "------------"
echo ""
echo "Memory Specs"
echo ""
free -mh

TODAY'S DATE:
Tue Dec 17 07:00:28 PST 2019
------------

LSB Version:	core-9.20170808ubuntu1-noarch:security-9.20170808ubuntu1-noarch
Distributor ID:	Ubuntu
Description:	Ubuntu 18.04.3 LTS
Release:	18.04
Codename:	bionic

------------
HOSTNAME: 
computer

------------
Computer Specs:

Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              4
On-line CPU(s) list: 0-3
Thread(s) per core:  2
Core(s) per socket:  2
Socket(s):           1
NUMA node(s):        1
Vendor ID:           GenuineIntel
CPU family:          6
Model:               58
Model name:          Intel(R) Core(TM) i7-3517U CPU @ 1.90GHz
Stepping:            9
CPU MHz:             1795.794
CPU max MHz:         3000.0000
CPU min MHz:         800.0000
BogoMIPS:            4788.78
Virtualization:      VT-x
L1d cache:           32K
L1i cache:           32K
L2 cache:            256K
L3 cache:            4096K
NUMA node0 CPU(s):   0-3
Flags:               fpu vme de pse

### Set variables¶
%env are best for bash

In [14]:
%env wd=/home/samb/analyses/20191214_olur_genome_feature_and_intron_splitting
wd="/home/samb/analyses/20191214_olur_genome_feature_and_intron_splitting"


# File download
%env downloads_list=Olurida_v081.fa.fai Olurida_v081_genome_snap02.all.renamed.putative_function.domain_added.gff Olurida_v081-20190709.exon.gff Olurida_v081-20190709.gene.gff
%env rsync_owl=owl:/volume1/web/halfshell/genomic-databank/

# Input/output files
%env chrome_sizes=Olurida_v081.sizes.txt
%env fa_index=Olurida_v081.fa.fai
%env maker_gff=Olurida_v081_genome_snap02.all.renamed.putative_function.domain_added.gff
%env base_name=Olurida_v081-20190709
%env exon_gff=Olurida_v081-20190709.exon.gff
%env exon_gff_sorted=Olurida_v081-20190709.exon.sorted.gff3
%env gene_gff=Olurida_v081-20190709.gene.gff
%env gene_gff_sorted=Olurida_v081-20190709.gene.sorted.gff3

%env exon_comp_bed=Olurida_v081-20190709.exon.sorted.comp.bed
%env intron_bed=Olurida_v081-20190709.introns.bed
%env intergenic_bed=Olurida_v081-20190709.intergenic.bed

# Programs
%env bedtools_dir=/home/samb/programs/bedtools-2.29.0/bin
%env samtools=/home/samb/programs/samtools-1.10

env: wd=/home/samb/analyses/20191214_olur_genome_feature_and_intron_splitting
env: downloads_list=Olurida_v081.fa.fai Olurida_v081_genome_snap02.all.renamed.putative_function.domain_added.gff Olurida_v081-20190709.exon.gff Olurida_v081-20190709.gene.gff
env: rsync_owl=owl:/volume1/web/halfshell/genomic-databank/
env: chrome_sizes=Olurida_v081.sizes.txt
env: fa_index=Olurida_v081.fa.fai
env: maker_gff=Olurida_v081_genome_snap02.all.renamed.putative_function.domain_added.gff
env: base_name=Olurida_v081-20190709
env: exon_gff=Olurida_v081-20190709.exon.gff
env: exon_gff_sorted=Olurida_v081-20190709.exon.sorted.gff3
env: gene_gff=Olurida_v081-20190709.gene.gff
env: gene_gff_sorted=Olurida_v081-20190709.gene.sorted.gff3
env: exon_comp_bed=Olurida_v081-20190709.exon.sorted.comp.bed
env: intron_bed=Olurida_v081-20190709.introns.bed
env: intergenic_bed=Olurida_v081-20190709.intergenic.bed
env: bedtools_dir=/home/samb/programs/bedtools-2.29.0/bin
env: samtools=/home/samb/programs/samtools-1.10


### Create necessary directories

In [5]:
%%bash
mkdir --parents ${wd}

In [6]:
cd {wd}

/home/samb/analyses/20191214_olur_genome_feature_and_intron_splitting


### Download files

In [11]:
%%bash

# Create array of files to download from list
mapfile -t downloads_array < <(echo ${downloads_list} | tr " " "\n")

for file in "${downloads_array[@]}"
do
  rsync \
  --archive \
  --verbose \
  "${rsync_owl}${file}" \
  .
## Uncomment lines below if need to use wget for downloads
# wget --quiet \
# --no-check-certificate \
# https://owl.fish.washington.edu/halfshell/genomic-databank/${file}
done
echo ""
echo ""
echo "----------------------------------------------------------"
echo ""



ls -lh

receiving incremental file list

sent 11 bytes  received 63 bytes  11.38 bytes/sec
total size is 6,103,215  speedup is 82,475.88
receiving incremental file list
Olurida_v081_genome_snap02.all.renamed.putative_function.domain_added.gff

sent 30 bytes  received 3,105,037,889 bytes  3,625,263.19 bytes/sec
total size is 3,104,658,743  speedup is 1.00
receiving incremental file list

sent 11 bytes  received 75 bytes  13.23 bytes/sec
total size is 9,248,086  speedup is 107,535.88


----------------------------------------------------------

total 3.0G
-rw-r--r-- 1 samb samb  16M Jul 16 07:10 Olurida_v081-20190709.exon.gff
-rw-r--r-- 1 samb samb 8.9M Jul 16 07:10 Olurida_v081-20190709.gene.gff
-rw-rw-rw- 1 samb samb 5.9M Aug 24  2018 Olurida_v081.fa.fai
-rw-rw-r-- 1 samb samb 2.9G Dec 13 21:30 Olurida_v081_genome_snap02.all.renamed.putative_function.domain_added.gff


X11 forwarding request failed on channel 0
X11 forwarding request failed on channel 0
Connection closed by 128.95.149.83 port 22
rsync: connection unexpectedly closed (0 bytes received so far) [Receiver]
rsync error: unexplained error (code 255) at io.c(235) [Receiver=3.1.2]
X11 forwarding request failed on channel 0


### Separate features

In [12]:
%%bash

# Create UTR array
utr_array=(five_prime_UTR three_prime_UTR)

echo "Here are the features and their counts in ${maker_gff}:"
awk 'NR>1 {print $3}' "${maker_gff}" | sort | uniq -c
echo ""
echo ""
echo "----------------------------------------------"


# Create GFFs
for feature in "${utr_array[@]}"
do
   output="${base_name}.${feature}.gff3"
   head -n 1 "${maker_gff}" >> "${output}"
   awk -v feature="$feature" '$3 == feature {print}' "${maker_gff}" \
   >> "${output}"
   echo ""
   echo ""
   echo "${output}"
   echo "----------------------------------------------"
   head -n 5 "${output}"
done

Here are the features and their counts in Olurida_v081_genome_snap02.all.renamed.putative_function.domain_added.gff:
19565118 
 163637 CDS
 159429 contig
 170394 exon
1214512 expressed_sequence_match
  14903 five_prime_UTR
  32210 gene
2099176 match
7027538 match_part
  32210 mRNA
 332676 protein_match
  13838 three_prime_UTR


----------------------------------------------


Olurida_v081-20190709.five_prime_UTR.gff3
----------------------------------------------
##gff-version 3
Contig58217	maker	five_prime_UTR	11504	11541	.	-	.	ID=OLUR_00019127-RA:five_prime_utr;Parent=OLUR_00019127-RA;
Contig9540	maker	five_prime_UTR	10078	10179	.	-	.	ID=OLUR_00018391-RA:five_prime_utr;Parent=OLUR_00018391-RA;
Contig36645	maker	five_prime_UTR	2185	2225	.	+	.	ID=OLUR_00022996-RA:five_prime_utr;Parent=OLUR_00022996-RA;
Contig3008	maker	five_prime_UTR	5726	6482	.	-	.	ID=OLUR_00018754-RA:five_prime_utr;Parent=OLUR_00018754-RA;


Olurida_v081-20190709.three_prime_UTR.gff3
---------------------------------

### Check exon and gene GFFs

In [15]:

%%bash
echo ${exon_gff}
head ${exon_gff}
echo ""
echo "----------------------"
echo ""
echo ${gene_gff}
head ${gene_gff}

Olurida_v081-20190709.exon.gff
##gff-version 3
Contig61093	maker	exon	7493	7773	.	+	.	ID=OLUR_00020575-RA:exon:1553;Parent=OLUR_00020575-RA;
Contig61093	maker	exon	7814	7946	.	+	.	ID=OLUR_00020575-RA:exon:1554;Parent=OLUR_00020575-RA;
Contig1111	maker	exon	24968	25494	.	-	.	ID=OLUR_00006628-RA:exon:209;Parent=OLUR_00006628-RA;
Contig1111	maker	exon	26015	26277	.	-	.	ID=OLUR_00006628-RA:exon:208;Parent=OLUR_00006628-RA;
Contig1111	maker	exon	26889	27080	.	-	.	ID=OLUR_00006628-RA:exon:207;Parent=OLUR_00006628-RA;
Contig1111	maker	exon	27651	27803	.	-	.	ID=OLUR_00006628-RA:exon:206;Parent=OLUR_00006628-RA;
Contig1111	maker	exon	28523	28696	.	-	.	ID=OLUR_00006628-RA:exon:205;Parent=OLUR_00006628-RA;
Contig214118	maker	exon	201	290	.	+	.	ID=OLUR_00032161-RA:exon:2219;Parent=OLUR_00032161-RA;
Contig214118	maker	exon	807	926	.	+	.	ID=OLUR_00032161-RA:exon:2220;Parent=OLUR_00032161-RA;

----------------------

Olurida_v081-20190709.gene.gff
##gff-version 3
Contig61093	maker	gene	7493	7946	.	+	

### Create chromosome sizes file

In [24]:
%%bash
# Change LC_COLLATE to use C for proper sorting of file
export LC_COLLATE=C

cut -f1,2 ${fa_index} | sort > ${chrome_sizes}

head ${chrome_sizes}

Contig0	116746
Contig1	87411
Contig10	79314
Contig100	58076
Contig1000	25824
Contig10000	6887
Contig100000	1518
Contig100001	5004
Contig100002	1810
Contig100003	1990


### Sort GFFs

In [20]:
%%bash
gff_array=("${exon_gff}" "${gene_gff}")
sorted_gff_array=("${exon_gff_sorted}" "${gene_gff_sorted}")

for index in "${!gff_array[@]}"
do
  { awk 'NR<2 {print}' "${gff_array[index]}"
   awk 'NR>1 {print}' "${gff_array[index]}" | "${bedtools_dir}"/bedtools sort -i -
  } >> "${sorted_gff_array[index]}"
  
  # Check out sorted GFFs
  echo "Previewing ${sorted_gff_array[index]}:"
  echo ""
  head "${sorted_gff_array[index]}"
  echo ""
  echo "Confirming sort order of ${sorted_gff_array[index]}:"
  echo ""
  cut -f1 "${sorted_gff_array[index]}" | uniq | head
  echo ""
done

Previewing Olurida_v081-20190709.exon.sorted.gff3:

##gff-version 3
Contig0	maker	exon	12497	12571	.	+	.	ID=OLUR_00000039-RA:exon:17;Parent=OLUR_00000039-RA;
Contig0	maker	exon	13175	13233	.	+	.	ID=OLUR_00000039-RA:exon:18;Parent=OLUR_00000039-RA;
Contig0	maker	exon	16209	16373	.	+	.	ID=OLUR_00000039-RA:exon:19;Parent=OLUR_00000039-RA;
Contig0	maker	exon	18857	18959	.	+	.	ID=OLUR_00000039-RA:exon:20;Parent=OLUR_00000039-RA;
Contig0	maker	exon	26493	26574	.	+	.	ID=OLUR_00000039-RA:exon:21;Parent=OLUR_00000039-RA;
Contig0	maker	exon	27806	27862	.	+	.	ID=OLUR_00000039-RA:exon:22;Parent=OLUR_00000039-RA;
Contig0	maker	exon	28510	28590	.	+	.	ID=OLUR_00000039-RA:exon:23;Parent=OLUR_00000039-RA;
Contig0	maker	exon	28735	28775	.	+	.	ID=OLUR_00000039-RA:exon:24;Parent=OLUR_00000039-RA;
Contig0	maker	exon	34306	34366	.	+	.	ID=OLUR_00000039-RA:exon:25;Parent=OLUR_00000039-RA;

Confirming sort order of Olurida_v081-20190709.exon.sorted.gff3:

##gff-version 3
Contig0
Contig1
Contig100
Contig1000
Co

### Create intergenic BED file

In [21]:
%%bash

"${bedtools_dir}"/complementBed \
-i "${gene_gff_sorted}" \
-g "${chrome_sizes}" \
> "${intergenic_bed}"

echo "Previewing ${intergenic_bed}:"
echo ""
head "${intergenic_bed}"

Previewing Olurida_v081-20190709.intergenic.bed:

Contig0	0	12496
Contig0	93068	95721
Contig0	104318	115730
Contig0	116730	116746
Contig1	0	7357
Contig1	13988	33347
Contig1	62707	74420
Contig1	78081	87411
Contig10	0	79314
Contig100	0	39283


Error: Sorted input specified, but the file Olurida_v081-20190709.gene.sorted.gff3 has the following out of order record
Contig0	maker	gene	12497	93068	.	+	.	ID=OLUR_00000039;Name=OLUR_00000039;Alias=maker-Contig0-snap-gene-0.8;Note=Similar to WDR87: WD repeat-containing protein 87 (Homo sapiens OX%3D9606);Dbxref=Coils:Coil,Gene3D:G3DSA:1.25.10.10,Gene3D:G3DSA:2.130.10.10,InterPro:IPR001680,InterPro:IPR011989,InterPro:IPR015943,InterPro:IPR016024,InterPro:IPR017986,InterPro:IPR036322,MobiDBLite:mobidb-lite,Pfam:PF00400,ProSiteProfiles:PS50082,ProSiteProfiles:PS50294,SMART:SM00320,SUPERFAMILY:SSF48371,SUPERFAMILY:SSF50978;Ontology_term=GO:0005515;


### Make intron BED file

In [22]:
%%bash
"${bedtools_dir}"/complementBed \
-i "${exon_gff_sorted}" \
-g "${chrome_sizes}" \
> "${exon_comp_bed}"

echo "Previewing ${exon_comp_bed}:"
echo ""
head "${exon_comp_bed}"
echo ""
echo "---------------------"

"${bedtools_dir}"/intersectBed \
-a "${gene_gff_sorted}" \
-b "${exon_comp_bed}" \
| awk -v OFS='\t' '{print $1,$4,$5}' \
> "${intron_bed}"

echo ""
echo "Previewing ${intron_bed}:"
echo ""
head "${intron_bed}"

Previewing Olurida_v081-20190709.exon.sorted.comp.bed:

Contig0	0	12496
Contig0	12571	13174
Contig0	13233	16208
Contig0	16373	18856
Contig0	18959	26492
Contig0	26574	27805
Contig0	27862	28509
Contig0	28590	28734
Contig0	28775	34305
Contig0	34366	35487

---------------------

Previewing Olurida_v081-20190709.introns.bed:

Contig0	12572	13174
Contig0	13234	16208
Contig0	18960	26492
Contig0	26575	27805
Contig0	27863	28509
Contig0	28591	28734
Contig0	34367	35487
Contig0	35641	36030
Contig0	36139	36864
Contig0	36941	42921


Error: Sorted input specified, but the file Olurida_v081-20190709.exon.sorted.gff3 has the following out of order record
Contig0	maker	exon	12497	12571	.	+	.	ID=OLUR_00000039-RA:exon:17;Parent=OLUR_00000039-RA;


### Cleanup

In [25]:
%%bash
rm "${maker_gff}" \
"${exon_comp_bed}" \
"${exon_gff}" \
"${exon_gff_sorted}" \
"${gene_gff}" \
"${gene_gff_sorted}" \
"${fa_index}"
ls -ltrh

total 16M
-rw-r--r-- 1 samb samb 1.6M Dec 17 07:44 Olurida_v081-20190709.five_prime_UTR.gff3
-rw-r--r-- 1 samb samb 1.5M Dec 17 07:44 Olurida_v081-20190709.three_prime_UTR.gff3
-rw-r--r-- 1 samb samb 3.7M Dec 17 07:51 Olurida_v081-20190709.intergenic.bed
-rw-r--r-- 1 samb samb 6.3M Dec 17 08:09 Olurida_v081-20190709.introns.bed
-rw-r--r-- 1 samb samb 2.7M Dec 17 08:15 Olurida_v081.sizes.txt
